# MLJ Example

The closest equivalent I've seen to `scikit-learn` for Julia is `MLJ`. `MLJ` provides a unified API wrapper around a variety of ML libraries available in Julia. 

Below let's take a look at 

In [95]:
# import Pkg; Pkg.add("MLJ"); Pkg.add("MLJDecisionTreeInterface")
using MLJ, DataFramesMeta, CairoMakie

In [4]:
# MLJ has some datasets builtin
iris = load_iris();
typeof(iris)

NamedTuple{(:sepal_length, :sepal_width, :petal_length, :petal_width, :target), Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, CategoricalArrays.CategoricalVector{String, UInt32, String, CategoricalArrays.CategoricalValue{String, UInt32}, Union{}}}}

In [7]:
# but it comes as a special NamedTuple with 'scitypes' inside of it, so let's make it into a dataframe
iris = DataFrame(iris)
# and we can `pretty` print the dataframe
first(iris,3) |> pretty

┌──────────────┬─────────────┬──────────────┬─────────────┬──────────────────────────────────┐
│ sepal_length │ sepal_width │ petal_length │ petal_width │ target                           │
│ Float64      │ Float64     │ Float64      │ Float64     │ CategoricalValue{String, UInt32} │
│ Continuous   │ Continuous  │ Continuous   │ Continuous  │ Multiclass{3}                    │
├──────────────┼─────────────┼──────────────┼─────────────┼──────────────────────────────────┤
│ 5.1          │ 3.5         │ 1.4          │ 0.2         │ setosa                           │
│ 4.9          │ 3.0         │ 1.4          │ 0.2         │ setosa                           │
│ 4.7          │ 3.2         │ 1.3          │ 0.2         │ setosa                           │
└──────────────┴─────────────┴──────────────┴─────────────┴──────────────────────────────────┘


In [97]:
# There's a univariate time series one here too
sunspots = MLJ.load_sunspots() |> DataFrame;

Let's run an initial model on this DataFrame. Since `MLJ` is a wrapper on top of a bunch of other libraries, we need a way to explore the catalog of models that are available and load specific models so that we can put them to work. 

* See the [Model Search](https://alan-turing-institute.github.io/MLJ.jl/dev/model_search/) section in the `MLJ` docs for more details

* `models("KMeans")`: Lists models in the catalog that match the name "KMeans"
* `info("KMeans", pkg="Clustering")`: Show the parameters for the "KMeans" model from the "Clustering" package
* `doc("KMeans", pkg="Clustering")`: Show the docs for the "KMeans" model from the "Clustering" package

In [146]:
# Show an example as a dataframe
example = info("KMeans", pkg="Clustering") |> pairs |> collect |> DataFrame
example[1,["name", "package_name", "human_name", "hyperparameters", "is_pure_julia"]] |> DataFrame |> pretty

┌─────────┬──────────────┬───────────────────┬──────────────────────────────────┬───────────────┐
│ name    │ package_name │ human_name        │ hyperparameters                  │ is_pure_julia │
│ String  │ String       │ String            │ Tuple{Symbol, Symbol, Symbol}    │ Bool          │
│ Textual │ Textual      │ Textual           │ Tuple{Unknown, Unknown, Unknown} │ Count         │
├─────────┼──────────────┼───────────────────┼──────────────────────────────────┼───────────────┤
│ KMeans  │ Clustering   │ K-means clusterer │ (:k, :metric, :init)             │ true          │
└─────────┴──────────────┴───────────────────┴──────────────────────────────────┴───────────────┘


Now here's an example of running the KMeans model with $k=3$ to split the `iris` data into 3 clusters:

In [200]:
KMeans = @load KMeans pkg=Clustering verbosity=0
# Split the dataframe into y and X, where y is the column `target`  and X is everything else
y, X = unpack(iris, ==(:target))
model = KMeans(k=3)
mach = machine(model, X) |> fit!

yhat = predict(mach, X)

# Check cluster assignments
compare = zip(yhat, y) |> collect
compare[1:8] # clusters align with classes

8-element Vector{Tuple{CategoricalArrays.CategoricalValue{Int64, UInt32}, CategoricalArrays.CategoricalValue{String, UInt32}}}:
 (1, "setosa")
 (1, "setosa")
 (1, "setosa")
 (1, "setosa")
 (1, "setosa")
 (1, "setosa")
 (1, "setosa")
 (1, "setosa")

Let's review what happened in the cell above:

* `@load`: loads the specified model from the catalog and brings it in to the environment
* `unpack(iris, ==(:target))`: horizontally split the `iris` data into X and y, where y are all columns called "target" (here we use the symbol `:target`)
* `KMeans(k=3)`: Instantiate the KMeans model (from the `@load` on line 1) and set $k=3$
*  `machine(model, X) |> fit!`: We create a 'machine' that combines a model with training data and then `fit!` trains the model on the data
   *  The pipe operator `|>` is similar to method chaining in python or `dplyr`'s pipe in R
*  `predict(mach, X)`: Provides the predicted cluster assignments for each row of $X$

This also created a fitted machine that we can call things like `fitted_params`, which in the case of `KMeans` just has a single parameter `centers` and each row corresponds to a column from the `iris` training data (sepal length, sepal width, etc)

In [222]:
fitted_params(mach).centers

4×3 Matrix{Float64}:
 5.006  5.88361  6.85385
 3.418  2.74098  3.07692
 1.464  4.38852  5.71538
 0.244  1.43443  2.05385

and we can generate a report on the fitted machine with `report` to get some of the key outputs. This includes:

* `assignments`: cluster assignments
* `cluster_labels`: cluster labels

In [213]:
report(mach)

(assignments = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  3, 3, 2, 3, 3, 3, 2, 3, 3, 2],
 cluster_labels = CategoricalArrays.CategoricalValue{Int64, UInt32}[1, 2, 3],)

## Regression

Let's build on the clustering example with an example from supervised learning. Below we'll take the popular boston housing dataset so we can show how to use `MLJ` to do things like a train/test split and evaluating against accuracy metrics.

In [224]:
# Load in boston housing market data
boston = load_boston() |> DataFrame
first(boston,3)

Row,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax,PTRatio,Black,LStat,MedV
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14,21.6
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7


In [227]:
# Train Test Split with `partition`
train, test = partition(boston, 0.8, rng=42);

# As before, unpack to horizontally split into y and X
y_train, X_train = unpack(train, ==(:MedV))
y_test, X_test = unpack(test, ==(:MedV));

# Load in our model and instantiate it
Tree = @load RandomForestRegressor pkg=DecisionTree verbosity=0;
tree = Tree()

RandomForestRegressor(
  max_depth = -1, 
  min_samples_leaf = 1, 
  min_samples_split = 2, 
  min_purity_increase = 0.0, 
  n_subfeatures = -1, 
  n_trees = 10, 
  sampling_fraction = 0.7, 
  feature_importance = :impurity, 
  rng = Random._GLOBAL_RNG())

Next we build our machine with the training data, and fit it:

In [247]:
mach = machine(tree, X_train, y_train) |> fit!

trained Machine; caches model-specific representations of data
  model: RandomForestRegressor(max_depth = -1, …)
  args: 
    1:	Source @537 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}
    2:	Source @931 ⏎ AbstractVector{Continuous}


### Model Evaluation

Now we want to check our accuracy in terms of RMSE - instead of doing the old `predict` one time, let's use `evaluate` to see how this model performs with 5-fold cross validation:

In [245]:
results = evaluate!(mach, resampling=CV(nfolds=5, rng=42), measure=rms, operation=predict)

PerformanceEvaluation object with these fields:
  measure, operation, measurement, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_rows
Extract:
┌────────────────────────┬───────────┬─────────────┬─────────┬──────────────────
│ measure                │ operation │ measurement │ 1.96*SE │ per_fold        ⋯
├────────────────────────┼───────────┼─────────────┼─────────┼──────────────────
│ RootMeanSquaredError() │ predict   │ 3.85        │ 0.613   │ [4.83, 3.62, 3. ⋯
└────────────────────────┴───────────┴─────────────┴─────────┴──────────────────
                                                                1 column omitted


This is really cool - it lets you specify the evaluation strategy quite nicely. 

* See [measure list](https://alan-turing-institute.github.io/MLJ.jl/stable/performance_measures/#List-of-measures) for a list of possible performance measures